# Feature Engineering

Welcome to this mini-course on feature engineering. The goal here is to help you learn a practical approach to feature engineering in as little time as possible. You'll be able to apply what you learn to Kaggle competitions as well as building machine learning applications. Through hands-on exercises you'll learn how to:

* Develop a baseline model for comparing performance on models with more features
* Encode categorical features so the model can make better use of the information
* Generate new features from the data to provide more information for the model
* Select features to reduce overfitting and increase the speed of making predictions

For the exercises, you'll be applying feature engineering techniques to data from the TalkingData AdTracking competition.

<center><a href="https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection"><img src="https://i.imgur.com/srKxEkD.png" width=600px></a></center>

The goal of the competition is to predict if a user will download an app after clicking through an ad. For this course I've taken a small sample of the data, dropping 99% of negative records (where the app wasn't downloaded) so that the target is more balanced.

### Kickstarter projects

To demonstrate feature engineering techniques, I'll be using data from Kickstarter projects. Then you'll apply what I show you in the tutorials to the TalkingData dataset. I'll load it up here and look at the first few rows.

In [1]:
import pandas as pd

In [2]:
ks = pd.read_csv('../input/kickstarter-projects/ks-projects-201801.csv',
                 parse_dates=['deadline', 'launched'])

In [3]:
ks.head(10)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.00,failed,0,GB,0.00,0.00,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.00,failed,15,US,100.00,2421.00,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.00,failed,3,US,220.00,220.00,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.00,failed,1,US,1.00,1.00,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.00,canceled,14,US,1283.00,1283.00,19500.00
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.00,successful,224,US,52375.00,52375.00,50000.00
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.00,successful,16,US,1205.00,1205.00,1000.00
7,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000.0,2016-02-01 20:05:12,453.00,failed,40,US,453.00,453.00,25000.00
8,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29,125000.0,2014-04-24 18:14:43,8233.00,canceled,58,US,8233.00,8233.00,125000.00
9,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10,65000.0,2014-07-11 21:55:48,6240.57,canceled,43,US,6240.57,6240.57,65000.00


What we can do here is predict if a Kickstarter project will succeed. We get the outcome from the `state` column. To predict the outcome we can use features such as category, currency, funding goal, country, and when it was launched. 

### Preparing target column

First I'll look at project states and convert the column into something we can use as targets in a model.

In [4]:
pd.unique(ks.state)

array(['failed', 'canceled', 'successful', 'live', 'undefined',
       'suspended'], dtype=object)

We have six states, how many records of each?

In [5]:
ks.groupby('state')['ID'].count()

state
canceled       38779
failed        197719
live            2799
successful    133956
suspended       1846
undefined       3562
Name: ID, dtype: int64

I don't want to go into too much data cleaning in this course, so here's what I'll do.

- Drop projects that are "live"
- Count "successful" states as `outcome = 1`
- Every other state is `outcome = 0`

In [6]:
# Drop live projects
ks = ks.query('state != "live"')

# Add outcome column, "successful" == 1, others are 0
ks = ks.assign(outcome=(ks['state'] == 'successful').astype(int))

### Converting timestamps

Next I'll convert the `launched` feature into categorical features we can use in a model. Since I loaded in the columns as timestamp data, I can get access to date and time values through the `.dt` attribute on the timestamp column.

In [7]:
ks = ks.assign(hour=ks.launched.dt.hour,
               day=ks.launched.dt.day,
               month=ks.launched.dt.month,
               year=ks.launched.dt.year)

ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome,hour,day,month,year
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,0,12,11,8,2015
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,0,4,2,9,2017
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,0,0,12,1,2013
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,0,3,17,3,2012
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,0,8,4,7,2015


### Prepping categorical variables

Now for the categorical variables -- `category`, `currency`, and `country` -- I'll need to convert them into integers so our model can use the data. For this I'll use scikit-learn's `LabelEncoder`. This assigns an integer to each value of the categorical feature and replaces those values with the integers.

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
cat_features = ['category', 'currency', 'country']
encoder = LabelEncoder()

# Apply the label encoder to each column
encoded = ks[cat_features].apply(encoder.fit_transform)
encoded.head(10)

,category,currency,country
0,108,5,9
1,93,13,22
2,93,13,22
3,90,13,22
4,55,13,22
5,123,13,22
6,58,13,22
7,41,13,22
8,113,13,22
9,39,13,22


I'll collect all the features we'll use in a new dataframe and use that to train a model.

In [10]:
# Since ks and encoded have the same index and I can easily join them
data = ks[['goal', 'hour', 'day', 'month', 'year', 'outcome']].join(encoded)

In [11]:
data.head()

,goal,hour,day,month,year,outcome,category,currency,country
0,1000.0,12,11,8,2015,0,108,5,9
1,30000.0,4,2,9,2017,0,93,13,22
2,45000.0,0,12,1,2013,0,93,13,22
3,5000.0,3,17,3,2012,0,90,13,22
4,19500.0,8,4,7,2015,0,55,13,22


### Creating training, validation, and test splits

Before training I need to create data sets for training, validation, and testing. I'll take a fairly simple approach here and split the data using slices. I'll use 10% of the data as a validation set, 10% for testing, and the other 80% for training.

In [12]:
valid_fraction = 0.1
valid_size = int(len(data) * valid_fraction)

train = data[:-2 * valid_size]
valid = data[-2 * valid_size:-valid_size]
test = data[-valid_size:]

In general you want to be careful that each data set has the same proportion of target classes. I'll print out the fraction of successful outcomes for each of our datasets.

In [13]:
for each in [train, valid, test]:
    print(f"Outcome fraction = {each.outcome.mean():.4f}")

Outcome fraction = 0.3570
Outcome fraction = 0.3539
Outcome fraction = 0.3542


This looks good, each set is around 35% true outcomes likely because the data was well randomized beforehand. A good way to do this automatically is with `sklearn.model_selection.StratifiedShuffleSplit` but I don't need to use it here.

### Training a LightGBM model

For this course we'll be using a LightGBM model. This is a tree-based model that typically provides the best performance, even compared to XGBoost. It's also relatively fast to train. We won't be doing much hyperparameter optimization because that isn't the goal of this course. So, our models won't be the absolute best performance you can get. But we'll be observing model performance as we improve the data through feature engineering.

In [ ]:
import lightgbm as lgb

feature_cols = train.columns.drop('outcome')

dtrain = lgb.Dataset(train[feature_cols], label=train['outcome'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['outcome'])
dtest = lgb.Dataset(test[feature_cols], label=test['outcome'])

param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10, verbose_eval=False)

### Making predictions & evaluating the model

Finally, let's make predictions on the test set with the model and see how well it performs. An important thing to remember is that you can overfit to the validation data. This is why we need a test set that the model never sees until the final evaluation.

In [15]:
from sklearn import metrics

In [16]:
ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['outcome'], ypred)

print(f"Test AUC score: {score}")

Test AUC score: 0.747615303004287


In the next exercise, you'll build a baseline model for the TalkingData dataset.